In [26]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import tikzplotlib
import timeit

In [27]:
# This section is just me figuring out how tensors work
#lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
#upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
#x = tf.random.uniform(shape=[1,100]) # Interior points
#training_pts = tf.concat([lower_bound, x, upper_bound], 1) # Actual training data
#x_0 = tf.slice(training_pts, [0,0], [1,1])
#x_1 = tf.slice(training_pts, [0,101], [1,1])
#print(x_0.numpy(), x_1.numpy())

In [28]:
# Construct Simple Feedforward Network
model = Sequential()
model.add(Dense(100, input_shape=[1], activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(1))
#model.compile(loss='mean_squared_error', optimizer='adam')

In [29]:
# Define Loss Function
def loss_fn(model,x):
    #print(x)
    with tf.GradientTape() as tt:
        tt.watch(x)
        with tf.GradientTape() as t:
            t.watch(x)
            f = model(x, training=True)
            #print('f = ', f)
        df_dx = t.gradient(f,x)
        #print(df_dx)
    d2f_dx2 = tt.gradient(df_dx,x)
    #print(d2f_dx2)
    
    u_0 = tf.zeros(shape=[1,1]) # Lower boundary condition
    u_1 = tf.ones(shape=[1,1]) # Upper boundary condition
    bound_weight = 1 # lambda = sqrt(bound_weight)
    mse = tf.keras.losses.MeanSquaredError()
    lower_bound_error = tf.math.subtract(tf.slice(f,[0,0],[1,1]),u_0)
    #print(lower_bound_error)
    upper_bound_error = tf.math.subtract(tf.slice(f,[99,0], [1,1]),u_1)
    loss_1 = tf.math.add(tf.nn.l2_loss(d2f_dx2), 
                       tf.nn.l2_loss(bound_weight*lower_bound_error))
    loss = tf.math.add(loss_1, tf.nn.l2_loss(bound_weight*upper_bound_error))
    return loss

In [30]:
def train_for():    
    # Train network
    optimizer = tf.keras.optimizers.Adam() # Fancy gradient decent
    epochs = 20
    train_loss_results = [] # For tracking loss during training
    iterations_per_epoch = 100
    minibatch_size = 100 # Number of points to be selected each iteration
    for epoch in range(epochs): # Uncomment to run for specific number of epochs
        epoch_loss_avg = tf.keras.metrics.Mean()
        for iteration in range(iterations_per_epoch):
            lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
            upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
            interior_pts = tf.random.uniform(shape=[1,minibatch_size-2]) # Interior points
            x = tf.transpose(tf.concat([lower_bound, interior_pts, upper_bound], 1)) # Actual training data
            with tf.GradientTape() as t:
                t.watch(x)
                #print(x)
                #f = model(x, training=True) # Estimate for u
                loss = loss_fn(model,x) # Loss
            grads = t.gradient(loss, model.trainable_weights) # Find model gradients
            optimizer.apply_gradients(zip(grads,model.trainable_weights)) # Perform gradient decent
            epoch_loss_avg.update_state(loss) # Track loss
            # End training iteration
        train_loss_results.append(epoch_loss_avg.result())
        #print("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))
        # End Epoch

In [31]:
def train_while():    
    # Train network
    optimizer = tf.keras.optimizers.Adam() # Fancy gradient decent
    epochs = 20
    train_loss_results = [] # For tracking loss during training
    iterations_per_epoch = 100
    minibatch_size = 100 # Number of points to be selected each iteration
    #for epoch in range(epochs): # Uncomment to run for specific number of epochs
    average_error = 1
    epoch = -1
    while average_error > 0.005: 
        epoch_loss_avg = tf.keras.metrics.Mean()
        for iteration in range(iterations_per_epoch):
            lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
            upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
            interior_pts = tf.random.uniform(shape=[1,minibatch_size-2]) # Interior points
            x = tf.transpose(tf.concat([lower_bound, interior_pts, upper_bound], 1)) # Actual training data
            with tf.GradientTape() as t:
                t.watch(x)
                #print(x)
                #f = model(x, training=True) # Estimate for u
                loss = loss_fn(model,x) # Loss
            grads = t.gradient(loss, model.trainable_weights) # Find model gradients
            optimizer.apply_gradients(zip(grads,model.trainable_weights)) # Perform gradient decent
            epoch_loss_avg.update_state(loss) # Track loss
            # End training iteration
        # Check average error    
        lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
        upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
        x = tf.sort(tf.random.uniform(shape=[1,98])) # Interior points
        test_points_tensor = tf.transpose(tf.concat([lower_bound, x, upper_bound], 1)) # Actual data
        test_points = test_points_tensor.numpy()
        g = model(test_points_tensor, training=False).numpy()
        average_error = np.sum(abs(g-test_points))/len(g)
        print(average_error)
        
        train_loss_results.append(epoch_loss_avg.result())
        epoch+=1 
        print("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))
        # End Epoch

In [25]:
# Run the model with real data
#train_for()
train_while()
lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
x = tf.sort(tf.random.uniform(shape=[1,98])) # Interior points
test_points_tensor = tf.transpose(tf.concat([lower_bound, x, upper_bound], 1)) # Actual data
test_points = test_points_tensor.numpy()
g = model(test_points_tensor, training=False).numpy()
x_actual = tf.sort(tf.random.uniform(shape=[1,1000]).numpy()) # Known solution
#print(test_points[0])
#print(x_actual[0])
#print(f[0])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('1D Laplace (DGM)')
ax.plot(x_actual[0],x_actual[0], label='u(x)', color='blue')
ax.plot(test_points,g,'r--', label='g(x)')
ax.legend()
ax.set_aspect('equal','box')

tikzplotlib.save("1d_laplace_dgm.tex")

average_error = np.sum(abs(g-test_points))/len(g)
print(average_error)

0.021919617652893065
Epoch 000: Loss: 0.035
0.015599018335342408
Epoch 001: Loss: 0.000
0.020328474044799805
Epoch 002: Loss: 0.000
0.02382408618927002
Epoch 003: Loss: 0.000
0.019614737033843994
Epoch 004: Loss: 0.000
0.014328228235244751
Epoch 005: Loss: 0.000
0.018713648319244384
Epoch 006: Loss: 0.000
0.015388003587722777
Epoch 007: Loss: 0.000
0.01914201259613037
Epoch 008: Loss: 0.000
0.01818466305732727
Epoch 009: Loss: 0.000
0.021039400100708008
Epoch 010: Loss: 0.000
0.0203191876411438
Epoch 011: Loss: 0.000
0.01920288324356079
Epoch 012: Loss: 0.000


KeyboardInterrupt: 

In [32]:
print(timeit.timeit(train_while, number=30))

0.002709064483642578
Epoch 000: Loss: 0.051
0.0037653157114982605
Epoch 000: Loss: 0.031
0.00016245562583208084
Epoch 000: Loss: 0.009
0.0005403127521276474
Epoch 000: Loss: 0.005
0.000595521442592144
Epoch 000: Loss: 0.006
0.0005954770743846893
Epoch 000: Loss: 0.003
7.335130125284195e-05
Epoch 000: Loss: 0.002
0.0007146549969911575
Epoch 000: Loss: 0.002
0.000710894912481308
Epoch 000: Loss: 0.002
0.0013414856791496277
Epoch 000: Loss: 0.002
0.0002593865245580673
Epoch 000: Loss: 0.002
0.00041046902537345887
Epoch 000: Loss: 0.001
0.0008210116624832154
Epoch 000: Loss: 0.001
0.00024222958832979203
Epoch 000: Loss: 0.004
0.0009955979138612748
Epoch 000: Loss: 0.001
6.963321939110756e-05
Epoch 000: Loss: 0.004
9.44211706519127e-05
Epoch 000: Loss: 0.001
0.0005196331441402435
Epoch 000: Loss: 0.001
0.0006967415660619736
Epoch 000: Loss: 0.002
0.0006572562456130982
Epoch 000: Loss: 0.001
0.00016473254188895226
Epoch 000: Loss: 0.002
0.0010318870842456818
Epoch 000: Loss: 0.000
0.00014494